In [1]:
# DON'T CHANGE this part: import libraries
import numpy as np
import scipy
import json
from nltk.stem import PorterStemmer 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
import re
import itertools

# 1. Preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming

In [2]:
# 1. preprocess: converting text to lowercase, coverting number, tokenization, removing stopword, stemming
def preprocess(text):
   # converting text to lowercase
    text = text.lower()

    # splitting text to words
    corpus = word_tokenize(text)

    # initialize stemming
    porter = PorterStemmer()

    # initialize stop words
    stop_words = set(stopwords.words('english'))

    # removing stop words and stemming and convert number to 'num'
    res = []
    for w in corpus:
        if not w in stop_words:
            if w.isdigit():
                res.append('num')
            else:
                res.append(porter.stem(w))

    return res

# Creating a matrix A with size (document_size, 1 + vocab_size) and label matrix b with size (document_size, 5)

In [3]:
def getAb(xs, ys):
    col1 = np.ones(xs.shape[0], dtype='float')
    A = np.insert(xs, 0, col1, axis=1)
    b = np.array(ys, dtype='float').reshape(len(ys), 5)
    return A, b

# Preparing the data set of file 'train.json' và 'valid.jason'

In [4]:
# DON'T CHANGE this part: read data path
# train_set_path, valid_set_path, random_number = input().split()
train_set_path, valid_set_path = 'train.json', 'valid.json'
random_number = 89

# Input data for label and text of train
label = []
reviewText = []
res = []
with open(train_set_path) as json_file:
    data = json.load(json_file)
    for p in data:
        
        # initializing label vector
        rate = [0, 0, 0, 0, 0]
        rate[int(p['overall'])-1] = 1
        label.append(rate)
        reviewText.append(p['reviewText'])
        
        # initializing word document vector
        res += preprocess(p['reviewText'])

In [5]:
# Input data for label and text of valid
label_valid = []
reviewText_valid = []
res_valid = []
with open(valid_set_path) as json_file:
    data = json.load(json_file)
    for p in data:
        
        # initializing label vector
        rate = [0, 0, 0, 0, 0]
        rate[int(p['overall'])-1] = 1
        label_valid.append(rate)
        
        # initializing word document vector
        reviewText_valid.append(p['reviewText'])

# Creating dictionary for all of key have the value 0, size of dictionary and the number of document

In [6]:
# Initializing dictionary
dictionary = {w: 0 for w in res}

# Vocab_size
vocab_size = len(dictionary)

# The number of document
document_size = len(reviewText)

# The number of valid document used for testing
document_valid_size = len(reviewText_valid)

# 2. Embedding: hitogram matrix:
ai = number of times word i appears in the document
$$ h = \frac{a}{1^{T}a} $$  h is word frequency or histogram vector

In [7]:
# Initializing histogram matrix of training data set with size (document_size, vocab_size)
histogram_matrix = np.empty((document_size, vocab_size), dtype='float')
for i in range(document_size):
    doc = dictionary.copy()
    review = preprocess(reviewText[i])
    
    # Creating word count vector
    for word in review:
        doc[word] += 1

    # Creating the histogram vector
    sum_doc = sum(doc.values())
    histogram = [doc[w]/sum_doc for w in doc]

    histogram_matrix[i] = histogram

In [8]:
# Initializing histogram matrix of valid data set with size (document_valid_size, vocab_size)
histogram_matrix_valid = np.empty((document_valid_size, vocab_size), dtype='float')
for i in range(document_valid_size):
    doc = dictionary.copy()
    review = preprocess(reviewText_valid[i])
    
    # Creating word count vector
    for word in review:
        if word in dictionary.keys():
            doc[word] += 1

    # Creating the histogram vector
    sum_doc = sum(doc.values())
    histogram = [doc[w]/sum_doc for w in doc]

    histogram_matrix_valid[i] = histogram

# 3. Classifier using linear regression: **$$\hat{x} = (A^{T}A)^{-1}A^{T}b = A^{+}b$$**

In [9]:
A, b = getAb(histogram_matrix, label)
x_hat = np.linalg.pinv(A) @ b

# 4. Accuracy (for metric): $$ acc = \frac{\sum(right)}{\sum(total)} $$

In [10]:
# Adding a column of 1 to matrix
col1 = np.ones(histogram_matrix_valid.shape[0], dtype='float')
A_valid = np.insert(histogram_matrix_valid, 0, col1, axis=1)

acc = 0.0
for index in range(document_valid_size):
    # Using softmax to return appropriate label
    final = scipy.special.softmax(A_valid[index][:] @ x_hat)
    
    # Counting the right 
    if(np.argmax(final) == np.argmax(label_valid[index])):
        acc += 1

# Accuracy = right / total
acc /= document_valid_size

print(preprocess(reviewText_valid[random_number]))
print(f"M2 - {acc}")

['great', 'photographi', 'class', ',', 'sadli', 'bought', 'wrong', 'kind', 'class', 'ask', '.', 'good', 'practic', 'paper']
M2 - 0.52
